In [17]:
class Pool:
    def __init__(self, x_reserve, y_reserve):
        self.reserves = [0] * 2
        self.reserves[0] = x_reserve
        self.reserves[1] = y_reserve
    
    def K(self):
        return self.reserves[0] * self.reserves[1]
    
    def out_amount(self, in_amount, in_reserve, out_reserve):
        in_with_fee = in_amount * 0.997
        return in_with_fee * out_reserve / (in_with_fee + in_reserve)
    
    def swap(self, in_index, in_amount):
        out_index = (in_index + 1) % 2
        
        in_reserve = self.reserves[in_index]
        out_reserve = self.reserves[out_index]
        out_amount = self.out_amount(in_amount, in_reserve, out_reserve)
        
        self.reserves[in_index] += in_amount
        self.reserves[out_index] -= out_amount
        
        return out_amount

In [18]:
pool = Pool(10, 10)

In [19]:
pool.swap(0, 1)

0.9066108938801491

In [20]:
10 - 100/ (10 + 0.997)


0.9066108938801491

In [21]:
pool = Pool(10, 10)

in_amount = 1
out_amount = 0

out_delta = pool.swap(0, in_amount)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

K: 100.02728016731837, out: 0.9066108938801491


In [22]:
pool = Pool(10, 10)

in_amount = 1
out_amount = 0

out_delta = pool.swap(0, in_amount / 2)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

out_delta = pool.swap(0, in_amount / 2)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

print("total out:", out_amount)

K: 100.01428775539362, out: 0.47482973758155933
K: 100.02792792738373, out: 0.43172226901992017
total out: 0.9065520066014795


In [24]:
pool = Pool(10, 10)

in_amount = 1
out_amount = 0

out_delta = pool.swap(0, in_amount / 3)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

out_delta = pool.swap(0, in_amount / 3)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

out_delta = pool.swap(0, in_amount / 3)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

print("total out:", out_amount)

K: 100.00967835596995, out: 0.32164403006742587
K: 100.01905514238955, out: 0.30156955033355554
K: 100.02814861044506, out: 0.2833183641040162
total out: 0.9065319445049975


In [29]:

in_amount = 1

for n in range(1, 10):
    pool = Pool(10, 10)
    out_amount = 0
    print("n:", n)
    print("")
    for _ in range(n):
        out_delta = pool.swap(0, in_amount / n)
        print(f"K: {pool.K()}, out: {out_delta}")
        out_amount += out_delta

    print("total out:", out_amount)
    print("")

n: 1

K: 100.02728016731837, out: 0.9066108938801491
total out: 0.9066108938801491

n: 2

K: 100.01428775539362, out: 0.47482973758155933
K: 100.02792792738373, out: 0.43172226901992017
total out: 0.9065520066014795

n: 3

K: 100.00967835596995, out: 0.32164403006742587
K: 100.01905514238955, out: 0.30156955033355554
K: 100.02814861044506, out: 0.2833183641040162
total out: 0.9065319445049975

n: 4

K: 100.0073176086055, out: 0.24318852598970658
K: 100.01446149871256, out: 0.23162466460909736
K: 100.02143973869433, out: 0.22086683370870044
K: 100.02825984732027, out: 0.2108418077542882
total out: 0.9065218320617926

n: 5

K: 100.00588269898229, out: 0.19550169617820656
K: 100.01165260201701, out: 0.18799324593554106
K: 100.01731395941094, out: 0.18090940133804922
K: 100.02287078556567, out: 0.17421873195878587
K: 100.02832687612253, out: 0.16789266312372766
total out: 0.9065157385343104

n: 6

K: 100.00491827466924, out: 0.163450661507943
K: 100.00975745648165, out: 0.15818571367125153